### Developed image classification of cifar10 dataset using pytorch 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#@title Import dependencies

import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import csv
from google.colab import files

### Defining the hyperparameters for model training

In [94]:
#@title Set hyperparameters

num_classes = 20   # number of output classes discrete range [0,9]
num_epochs  = 30    # number of times the entire dataset is presented to the model
batch_size  = 64   # the size of input data took for one iteration
lr          = 0.002 # size of step
momentum = 0.9
weight_decay=5e-4
nesterov=True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [95]:
#@title Download CIFAR-10 and set up dataloaders

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

train_data = dsets.CIFAR10(root='./data',
                           train=True,
                           transform=transform, download=True)

test_data = dsets.CIFAR10(root='./data', train=False,
                          transform=transform, download=True)

train_gen = torch.utils.data.DataLoader(dataset=train_data,
                                        batch_size=batch_size,
                                        shuffle=True, num_workers=2)

test_gen = torch.utils.data.DataLoader(dataset=test_data,
                                       batch_size=batch_size, 
                                       shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### Defining the model.
 Model is created with 3 convolutional layers

In [0]:
#@title Define the model
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self, num_classes):
    super(Net, self).__init__()
    ############################################################################
    # TODO (optional): Define modules you might need for your neural network.  #
    ############################################################################
    self.conv_layer = nn.Sequential(
      # Conv Layer block 1
      nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(inplace=True),
      nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2, stride=2),

      # Conv Layer block 2
      nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(inplace=True),
      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Dropout2d(p=0.05),

      # Conv Layer block 3
      nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
      nn.BatchNorm2d(256),
      nn.ReLU(inplace=True),
      nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2, stride=2),

    )

    self.fc_layer = nn.Sequential(
        nn.Dropout(p=0.1),
        nn.Linear(4096, 1024),
        nn.ReLU(inplace=True),
        nn.Linear(1024, 512),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.1),
        nn.Linear(512, 10)
    )

    ############################################################################
    #                              END OF YOUR CODE                            #
    ############################################################################

  def forward(self, x):
    ############################################################################
    # TODO: Perform a forward pass through your network from inputs `x` of     #
    # shape (batch_size x 32 x 32) to output class scores `out` of shape       #
    # (batch_size x 10). You can use modules defined in the constructor above. #
    ############################################################################
    # conv layers
    x = self.conv_layer(x)
    
    # flatten
    x = x.view(x.size(0), -1)
    
    # fc layer
    x = self.fc_layer(x)

    return x

    ############################################################################
    #                              END OF YOUR CODE                            #
    ############################################################################
    return out


In [97]:
#@title Build the model

net = Net(num_classes)
net.to(device)

Net(
  (conv_layer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Dropout2d(p=0.05, inplace=False)
    (13): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [0]:
#@title Define loss function & optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4, nesterov=True)

In [99]:
#@title Train model

net.train()
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_gen):
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()
    outputs = net(images)

    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size,
                   loss.item()))

Epoch [1/30], Step [100/781], Loss: 2.0894
Epoch [1/30], Step [200/781], Loss: 1.6813
Epoch [1/30], Step [300/781], Loss: 1.5420
Epoch [1/30], Step [400/781], Loss: 1.3253
Epoch [1/30], Step [500/781], Loss: 1.3803
Epoch [1/30], Step [600/781], Loss: 1.2281
Epoch [1/30], Step [700/781], Loss: 1.1619
Epoch [2/30], Step [100/781], Loss: 1.0442
Epoch [2/30], Step [200/781], Loss: 0.8567
Epoch [2/30], Step [300/781], Loss: 0.8320
Epoch [2/30], Step [400/781], Loss: 0.7931
Epoch [2/30], Step [500/781], Loss: 0.8361
Epoch [2/30], Step [600/781], Loss: 0.7413
Epoch [2/30], Step [700/781], Loss: 0.7732
Epoch [3/30], Step [100/781], Loss: 0.7789
Epoch [3/30], Step [200/781], Loss: 0.8998
Epoch [3/30], Step [300/781], Loss: 0.5946
Epoch [3/30], Step [400/781], Loss: 0.7189
Epoch [3/30], Step [500/781], Loss: 0.6387
Epoch [3/30], Step [600/781], Loss: 0.6426
Epoch [3/30], Step [700/781], Loss: 0.8161
Epoch [4/30], Step [100/781], Loss: 0.5312
Epoch [4/30], Step [200/781], Loss: 0.6191
Epoch [4/30

### Save the model

In [0]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [101]:
net = Net(10)
net.cuda()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

### Testing the accuracy of classes

In [104]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_gen:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 91 %
Accuracy of   car : 96 %
Accuracy of  bird : 81 %
Accuracy of   cat : 63 %
Accuracy of  deer : 72 %
Accuracy of   dog : 81 %
Accuracy of  frog : 85 %
Accuracy of horse : 84 %
Accuracy of  ship : 91 %
Accuracy of truck : 91 %


### Write predictions to caliper_cifar10_test_predictions.csv and downloading the file.

In [0]:
#@title Run model on test set and save predictions

net.eval()
predictions = []

for images, _ in test_gen:
  images = images.to(device)

  output = net(images)
  _, predicted = torch.max(output, 1)
    
  img_id_start = len(predictions) + 1
  predictions += [{'image_id': img_id_start + x,
                   'label': predicted[x].item()} for x in range(len(predicted))]

with open('gdrive/My Drive/Data/caliper_cifar10_test_predictions.csv', mode='w') as preds_file:
    writer = csv.writer(preds_file, delimiter=',')
    writer.writerow(['id','label'])
    for el in predictions:
        writer.writerow([el['image_id'], el['label']])
    
files.download('gdrive/My Drive/Data/caliper_cifar10_test_predictions.csv')